# DeepAR

In [17]:
import pandas as pd
import numpy as np
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.dataset.common import ListDataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator
from datetime import datetime, timedelta
import yfinance as yf
import matplotlib.pyplot as plt

In [7]:
prediction_length = 10
end_date = datetime.today()
start_date = end_date - timedelta(days=365)  
apple_stock_price = yf.download("AAPL", start=start_date, end=end_date, interval="1d")["Adj Close"]

[*********************100%%**********************]  1 of 1 completed


In [8]:

apple_stock_price = apple_stock_price.reset_index()
apple_stock_price.columns = ['ds', 'y']

df = apple_stock_price

In [9]:
# data has to be in a specific format for the GluonTS package

train_ds = ListDataset(
    [{"start": df['ds'].iloc[0], "target": df['y'].values}],
    freq="D"
)

In [10]:
estimator = DeepAREstimator(
    freq="D",
    prediction_length=prediction_length,  # The forecast horizon
    trainer=Trainer(epochs=20)
)

In [11]:
predictor = estimator.train(training_data=train_ds)

100%|██████████| 50/50 [00:01<00:00, 37.08it/s, epoch=20/20, avg_epoch_loss=2.61]


In [12]:
test_ds = ListDataset(
    [{"start": df['ds'].iloc[0], "target": df['y'].values}],
    freq="D"
)

In [13]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds, predictor=predictor, num_samples=100
)

In [14]:
forecasts = list(forecast_it)
tss = list(ts_it)

# Evaluate the model
evaluator = Evaluator()
agg_metrics, item_metrics = evaluator(tss, forecasts)

print("Aggregate metrics:\n", agg_metrics)
print("Item metrics:\n", item_metrics)

Running evaluation: 1it [00:00, 62.10it/s]

Aggregate metrics:
 {'MSE': 100.5635986328125, 'abs_error': 85.50347900390625, 'abs_target_sum': 2072.1201171875, 'abs_target_mean': 207.21201171875, 'seasonal_error': 1.7442537943522136, 'MASE': 4.902009058587761, 'MAPE': 0.04042007029056549, 'sMAPE': 0.041521936655044556, 'MSIS': 72.6485756380414, 'num_masked_target_values': 0.0, 'QuantileLoss[0.1]': 25.76131591796875, 'Coverage[0.1]': 0.0, 'QuantileLoss[0.2]': 44.4538818359375, 'Coverage[0.2]': 0.0, 'QuantileLoss[0.3]': 60.654867553710936, 'Coverage[0.3]': 0.0, 'QuantileLoss[0.4]': 74.06569213867188, 'Coverage[0.4]': 0.1, 'QuantileLoss[0.5]': 85.50347900390625, 'Coverage[0.5]': 0.1, 'QuantileLoss[0.6]': 95.39305419921877, 'Coverage[0.6]': 0.1, 'QuantileLoss[0.7]': 100.7367950439453, 'Coverage[0.7]': 0.2, 'QuantileLoss[0.8]': 102.04116210937502, 'Coverage[0.8]': 0.2, 'QuantileLoss[0.9]': 88.72214965820312, 'Coverage[0.9]': 0.4, 'RMSE': 10.028140337710303, 'NRMSE': 0.04839555513471658, 'ND': 0.04126376569325556, 'wQuantileLoss[0.1]': 

In [19]:
def plot_forecast(tss, forecasts, prediction_length):
    for target, forecast in zip(tss, forecasts):
        plt.figure(figsize=(12, 6))
        target_index = pd.date_range(start=target.start_date, periods=len(target), freq='D')
        plt.plot(target_index, target, label="Actual")
        forecast_index = pd.date_range(start=target_index[-1] + pd.Timedelta(days=1), periods=prediction_length, freq='D')
        plt.plot(forecast_index, forecast.mean, label="Forecast")
        plt.fill_between(forecast_index, forecast.quantile(0.1), forecast.quantile(0.9), color='r', alpha=0.3)
        plt.legend()
        plt.show()

# Plot the forecast
plot_forecast(tss, forecasts, prediction_length)

AttributeError: 'DataFrame' object has no attribute 'start_date'

<Figure size 1200x600 with 0 Axes>